In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.display import clear_output
import pandas as pd
import random

In [2]:
df_urls = pd.read_csv('urls_propiedades_en_venta_san_martin_argenprop', index_col=False)
urls = df_urls['0'].tolist()

In [3]:
%%time

features_global = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
localidades = ['centro','ballester','suarez','martin', 'maipu','lynch','andres','billinghurst','malaver',
              'bonich','hermosa','hidalgo','progreso','golf', 'chilavert', 'parque']

for url in urls[862 :]:
    features_local = {}
    driver.get(url)
    
    # Extraccion del precio en USD
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'titlebar__price').text
        element = element.replace('USD', '')
        element = element.replace('.', '')
        element = element.replace(' ', '')
        features_local['precio_USD'] = int(element)
    except:
        pass
    
    # Categoria de la propiedad (casa, departamento o ph)
    
    if 'casa' in url:
        features_local['categoria'] = 'casa'
    elif 'ph' in url: 
        features_local['categoria'] = 'ph'
    elif 'departamento' in url:
        features_local['categoria'] = 'departamento'
        
    # Extracción de la localidad
    
    for loc in localidades:
        if loc in url:
            features_local['localidad'] = loc
    
    # Extraccion de los elementos principales
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'property-main-features')
        elements = element.find_elements(By.TAG_NAME, 'p')
    except:
        pass
    
    try:
        features_local[elements[0].text] = elements[1].text
    except:
        pass

    try:
        features_local[elements[2].text] = elements[3].text
    except:
        pass

    try:
        features_local[elements[4].text] = elements[5].text
    except:
        pass

    try:
        features_local[elements[6].text] = elements[7].text
    except:
        pass

    try:
        features_local[elements[8].text] = elements[9].text
    except:
        pass

    # Correcion de elementos que son equivalentes, son inecesarios o hay que formatear.
     
    try:
        features_local['Baños'] = features_local['Cantidad de baños']
        del features_local['Cantidad de baños']
    except:
        pass

    try:
        features_local['Baños'] = int(float(features_local['Baños']))
    except:
        pass
    
    try:
        features_local['Dormitorios'] = features_local['Cantidad de dormitorios']
        del features_local['Cantidad de dormitorios']
    except:
        pass
    
    try:
        features_local['Dormitorios'] = features_local['Dormitorios'].replace('Monoambiente', '0')
        features_local['Dormitorios'] = int(float(features_local['Dormitorios']))
    except:
        pass
    
    try:
        features_local['Antiguedad'] = features_local['Antigüedad']
        del features_local['Antigüedad']
    except:
        pass
    
    try:
        features_local['Antiguedad'] = features_local['Antiguedad'].replace('A Estrenar', '0')
        features_local['Antiguedad'] = features_local['Antiguedad'].replace('A estrenar', '0')
        features_local['Antiguedad'] = features_local['Antiguedad'].replace(' años', '')
        features_local['Antiguedad'] = features_local['Antiguedad'].replace(' año', '')
        features_local['Antiguedad'] = features_local['Antiguedad'].replace(',', '.')
        features_local['Antiguedad'] = int(float(features_local['Antiguedad']))
    except:
        pass
    
    try:
        features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace('m²', '')
        features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(' ', '')
        features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(',', '.')
        features_local['Superficie cubierta'] = float(features_local['Superficie cubierta'])
        features_local['Superficie cubierta'] = int(features_local['Superficie cubierta'])
    except:
        pass
    
    try:
        del features_local['Disposición']
    except:
        pass
    
    # Extracion de la latitud y la longitud
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'map-container')
        element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-latitude')
        features_local['latitud'] = float(element.replace(',','.'))
    except:
        pass
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'map-container')
        element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-longitude')
        features_local['logitud'] = float(element.replace(',','.'))
    except:
        pass
    
    
    features_global.append(features_local)
    
    clear_output(wait=True)
    
    print(f'Propiedad número {urls.index(url) + 1} extraída ({round(((urls.index(url) + 1)/len(urls))*100 , 2)}%)')
    
    df = pd.DataFrame(features_global)
    
    df = df.rename(columns={'Superficie cubierta':'superficie_cubierta_m2',
                        'Dormitorios':'dormitorios',
                        'Baños':'banos',
                        'Antiguedad':'antiguedad'})
    
    df.to_csv('features_propiedades.csv')
    
driver.close()

Propiedad número 863 extraída (10.65%)


WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=102.0.5005.115)
Stacktrace:
#0 0x55c8e192df33 <unknown>
#1 0x55c8e1678118 <unknown>
#2 0x55c8e1671f25 <unknown>
#3 0x55c8e16637a6 <unknown>
#4 0x55c8e16646bb <unknown>
#5 0x55c8e1663a72 <unknown>
#6 0x55c8e1662f15 <unknown>
#7 0x55c8e1661c0f <unknown>
#8 0x55c8e1662042 <unknown>
#9 0x55c8e1679b92 <unknown>
#10 0x55c8e16dfa3f <unknown>
#11 0x55c8e16cbf72 <unknown>
#12 0x55c8e16df2e4 <unknown>
#13 0x55c8e16cbe63 <unknown>
#14 0x55c8e16a182a <unknown>
#15 0x55c8e16a2985 <unknown>
#16 0x55c8e19724cd <unknown>
#17 0x55c8e19765ec <unknown>
#18 0x55c8e195c71e <unknown>
#19 0x55c8e1977238 <unknown>
#20 0x55c8e1951870 <unknown>
#21 0x55c8e1993608 <unknown>
#22 0x55c8e1993788 <unknown>
#23 0x55c8e19adf1d <unknown>
#24 0x7f9d271ff609 <unknown>


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   categoria               863 non-null    object 
 1   localidad               859 non-null    object 
 2   superficie_cubierta_m2  767 non-null    float64
 3   dormitorios             828 non-null    float64
 4   antiguedad              683 non-null    float64
 5   banos                   803 non-null    float64
 6   precio_USD              851 non-null    float64
 7   latitud                 795 non-null    float64
 8   logitud                 795 non-null    float64
dtypes: float64(7), object(2)
memory usage: 60.8+ KB
